In [1]:
import pandas as pd
import numpy as np
from itertools import permutations

## 1. Recommendation sytem
Implementing a recommendation system is critical for businesses and digital platforms that want to thrive in today's competitive environment. These systems use data-driven personalization to tailor content, products, and services to individual user preferences. The latter improves user engagement, satisfaction, retention, and revenue through increased sales and cross-selling opportunities. In this section, you will attempt to implement a recommendation system by identifying similar users' preferences and recommending movies they watch to the study user.

To be more specific, you will implement your version of the [**LSH algorithm**](https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/), which will take as input the user's preferred genre of movies, find the most similar users to this user, and recommend the most watched movies by those who are more similar to the user.

__Data__: The data you will be working with can be found [here](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies).

Looking at the data, you can see that there is data available for each user for the movies the user <ins>clicked on</ins>. Gather the __title and genre__ of the __maximum top 10 movies__ that each user clicked on regarding the __number of clicks__.

---


### Pre-processing the data frame

Firstly, let's import the data. We can observe that the dataset is in CSV format, making it straightforward to import.

In [3]:
# Save the csv file in a variable to work on it
df = pd.read_csv("./uk_movies_03.csv")

We can use .info() to obtain a concise summary of information about the DataFrame.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671736 entries, 0 to 671735
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    671736 non-null  int64  
 1   datetime      671736 non-null  object 
 2   duration      671736 non-null  float64
 3   title         671736 non-null  object 
 4   genres        671736 non-null  object 
 5   release_date  671736 non-null  object 
 6   movie_id      671736 non-null  object 
 7   user_id       671736 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 41.0+ MB


We have a dataset with 7 columns, where each one of the 671 736 row corresponds to a user click on a specific movie. The users are identified by the `user_id` column, and each click is associated with a particular film denoted by the `movie_id` column.

We can also notice that there no null values. Now let's see if there are duplicates among the entries.

In [5]:
# analyze if there are duplicates
sum(df.duplicated())

0

All the rows are unique, so we don't need to do other manipulations on the dataframe.

### EXERCISE 1.1

Let's proceed by retrieving the titles and genres of the top 10 movies that each user clicked on, based on the number of clicks.

The approach involves grouping the dataframe by both `user_id` and `movie_id` and counting the occurrences of the same `movie_id` for each user. This new info will be stored in the `clicks` column.

In [6]:
# group the df by user_id and movie_id and create another column clicks
movie_counts = df.groupby(['user_id', 'movie_id']).size().reset_index(name='clicks')
# sort the df by clicks for every user
df_sorted = movie_counts.sort_values(by=['user_id', 'clicks'], ascending=[True, False])
# let's consider only the first 10 films for each user
top_10_movies_per_user = df_sorted.groupby('user_id').head(10)
top_10_movies_per_user

,user_id,movie_id,clicks
0,00004e2862,9bfee795ff,1
2,000052a0a0,4718f9963c,9
3,000052a0a0,4fa0b092d6,3
6,000052a0a0,7314699c23,3
5,000052a0a0,6275614f9a,2
...,...,...,...
502501,fffeac83be,a2d8a56924,1
502503,fffeac83be,dda0eae17b,1
502504,ffff2c5f9e,6467fee6b6,1
502505,ffff2c5f9e,9ab62a3f2c,1


This new dataframe currently contains only the movie IDs and it lacks the information about movie titles and genres. To enrich this information, we can merge it with the original dataframe.

In [7]:
# get the title and the genres for each movie_id
res = pd.merge(top_10_movies_per_user[['user_id', 'movie_id', 'clicks']],
            df[['movie_id', 'title', 'genres', 'user_id']], on=['movie_id', 'user_id'], how = "left")

# drop duplicates generated during the merge operation
res.drop_duplicates()

,user_id,movie_id,clicks,title,genres
0,00004e2862,9bfee795ff,1,Hannibal,"Crime, Drama, Thriller"
1,000052a0a0,4718f9963c,9,Looper,"Action, Drama, Sci-Fi, Thriller"
10,000052a0a0,4fa0b092d6,3,Jumanji,"Adventure, Comedy, Family, Fantasy"
13,000052a0a0,7314699c23,3,Frailty,"Crime, Drama, Thriller"
16,000052a0a0,6275614f9a,2,Resident Evil,"Action, Horror, Sci-Fi"
...,...,...,...,...,...
613389,fffeac83be,a2d8a56924,1,Son of Saul,"Drama, War"
613390,fffeac83be,dda0eae17b,1,Enemy at the Gates,"Drama, History, War"
613391,ffff2c5f9e,6467fee6b6,1,Hot Fuzz,"Action, Comedy, Mystery, Thriller"
613392,ffff2c5f9e,9ab62a3f2c,1,Forks Over Knives,Documentary


If we want to get the top 10 movies clicked in general by all the users, not the top 10 for every user, we can use the following code, which is implemented in a manner similar to the previous approach.

In [8]:
grouped_df = df.groupby(['title', 'genres']).size().reset_index(name='clicks').reset_index(drop=True)
grouped_df = grouped_df.sort_values(by='clicks',ascending=False)
grouped_df.head(10)

,title,genres,clicks
1035,Black Mirror: Bandersnatch,"Drama, Mystery, Sci-Fi, Thriller",6436
1207,Bright,"Action, Fantasy, Thriller",3110
705,Avengers: Age of Ultron,"Action, Adventure, Sci-Fi",2898
589,Annihilation,"Adventure, Drama, Horror, Mystery, Sci-Fi, Thr...",2699
2967,Hot Fuzz,"Action, Comedy, Mystery, Thriller",2674
1761,Deadpool,"Action, Adventure, Comedy, Sci-Fi",2576
1018,Bird Box,"Drama, Horror, Sci-Fi",2549
2221,FYRE: The Greatest Party That Never Happened,"Documentary, Music",2332
6399,The Big Short,"Biography, Comedy, Drama, History",2204
6826,The Hitman's Bodyguard,"Action, Comedy, Crime, Thriller",2179


### 1.2 Minhash Signatures
Using the movie `genre` and `user_ids`, try to implement your min-hash signatures so that users with similar interests in a genre appear in the same bucket.

__Important note:__ You must write your minhash function from scratch.  You are not permitted to use any already implemented hash functions.  Read the class materials and, if necessary, conduct an internet search.  The description of hash functions in the [book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf) may be helpful as a reference.

---

We need to implement the _MinHash function_, to estimate the similarity between an input user and all the others, in order to detect similar users and recommend films that could like.

In this initial phase of our implementation of it, we aim to create MinHash signatures for users based on their genre preferences and subsequently organize them into different buckets to group users with similar genre interests.

The key steps for implementing the MinHash function can be summarized as follows:

1. **Shingle Matrix**: in this initial step, we'll construct the shingle matrix by utilizing the genres observed by users. The shingle matrix will have unique genres as columns and user IDs as row indexes. For each entry in the matrix, a value of 1 will indicate that the user has viewed a film associated with that particular genre, we will put a 0 otherwise.

In [9]:
# Let's get the list of the unique genres
genres = df["genres"].to_frame() # extract the 'genres' column

uni = [] # list where we will put the genres

for i in range(len(genres)):
    x=genres["genres"][i].split(', ')
    for j in x:
        uni.append(j)

uni = set(uni) # transform 'uni' as a set to obtain unique values
uni # set of unique genres

{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'NOT AVAILABLE',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

We can notice the presence of the 'NOT AVAILABLE' genre, which does not contribute to our implementation, so we will not consider it when creating the new column `genres_list` with the converted genres from comma separated value, to a list of values.

In [10]:
df['genres_list'] = df['genres'].apply(lambda row: [word.strip() for word in row.split(',') if word.strip() != 'NOT AVAILABLE'])
df.head(2)

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id,genres_list
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe,"[Comedy, Drama, Romance]"
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510,"[Fantasy, Horror, Mystery, Thriller]"


We are now ready to construct a dictionary where each user ID serves as a key, and the corresponding value is the list of unique genres associated with that user. This dictionary encapsulates the diverse genre preferences of each user, it will be useful later.

In [11]:
# Initialize an empty dictionary to store user genres
user_genres = {}

# Iterate through each row in the DataFrame
for i in range(len(df)):
    # Extract user ID and genres list from the current row
    user_id = df["user_id"][i]
    genres_list = set(df["genres_list"][i])

    # Check if the user ID is already in the dictionary
    if user_id not in user_genres:
        # If not, create a new entry with the current genres list
        user_genres[user_id] = genres_list
    else:
        # If the user ID already exists, update the existing genres list with the current genres
        user_genres[user_id].update(genres_list)


Now, we can proceed to construct our shingle matrix, as outlined earlier, initializing it with NaN values.

Notably, the matrix will consist of 161 918 rows, corresponding to the unique user IDs in our dataset.

In [12]:
list_unique_genres = list(uni) # list of unique genres
matrix = pd.DataFrame(index = user_genres.keys(), columns = sorted(list_unique_genres)) # initial shingle matrix
matrix

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
1dea19f6fe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544dcbc510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7cbcc791bf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ebf43c36b6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
a57c992287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45414be0ec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
783ec67e84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89c715f3a4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9207e1499b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now let's populate the matrix. We will put a 1 if the user has seen a movie regarding a specific genre, 0 otherwise.

In [13]:
# we can iterate through the dictionary defined before
for user, genres in user_genres.items():
    for g in genres:
        matrix.loc[str(user)][g] = 1 # put 1 in the matrix if user_genres[user] contains the specific genre

# let's replace the NaN values with 0
matrix = matrix.fillna(0)
matrix

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
1dea19f6fe,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
544dcbc510,0,1,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
7cbcc791bf,1,1,1,0,1,1,0,0,1,1,...,0,0,1,0,0,0,0,1,0,0
ebf43c36b6,1,1,1,1,1,1,0,1,1,1,...,0,0,0,1,0,0,0,1,0,0
a57c992287,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45414be0ec,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
783ec67e84,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
89c715f3a4,0,1,1,0,1,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
9207e1499b,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


The matrix depicted above is the **shingle matrix**, and it's easy to understand that it is sparse. In other words, there is a substantial prevalence of 0s compared to 1s. This is a problem when we have to compute similarity between users, because the 0s essentially are useless, while the presence of the 1s is the only important information. The issue is that it could be computationally very expensive computing similarities using this matrix and storage data in this way, but it's useful to visualize what we are going to do.

2. **Signature Matrix**: this step is essential in order to solve the issue exponed before. In fact it is used to store data in a simple way and more efficient way than before. To create the signature matrix we need to perform a certain number of hash functions; each hash function should map elements from the row to a hash value, in order to not have 0 but only useful values in the new matrix and compute similarity in a more efficient way.

We decided to follow book's instruction and create the hash function as a row permutation. The procedure will be:
- Randomly permute the rows of the shingle matrix.
- For each row, start from the first column and find the position (index) of the first 1 (genre) that appears in the row.
- Use this value to represent the row.
- Repeat the permutation k times

Let's apply what we have said to our shingle matrix, firstly creating the permutations.

In [14]:
# Let's start from creating a dictionary with all the user ids
# where we can store the values of our new signature matrix

permutation = dict()

for i in user_genres.keys():
    permutation[i] = list() # initialize every user (key) of the dictionary with an empty list

num_hash = 12 # number of permutations

for i in range(num_hash):
    column_permutation = np.random.permutation(matrix.columns) # create random permutation of the rows of the dataframe
    df_permuted = matrix[column_permutation] # df permuted
    indx = np.argmax(df_permuted, axis=1) + 1   # find the index of the first 1 occurrence of every row,
                                                # adding 1 because otherwise row 1 would be indexed as 0.
    # populate the dictionary with users and indexes
    for count, user in enumerate(user_genres):
        permutation[user].append(indx[count])

signature_matrix = pd.DataFrame(permutation) # let's transform permutation in a dataframe
signature_matrix

,1dea19f6fe,544dcbc510,7cbcc791bf,ebf43c36b6,a57c992287,c5bf4f3f57,8e1be40e32,892a51dee1,cff8ea652a,bf53608c70,...,8a73e54068,6c9340966f,0f7e82f3bd,5301342a0a,5463651e3a,45414be0ec,783ec67e84,89c715f3a4,9207e1499b,57501964fd
0,6,1,1,1,1,1,1,8,1,2,...,10,2,2,1,10,24,3,2,6,17
1,4,4,4,1,1,4,3,1,1,1,...,15,4,5,4,15,4,2,1,3,22
2,8,1,1,1,1,1,1,1,7,1,...,9,12,1,14,9,8,2,1,2,6
3,10,1,3,2,1,3,1,2,2,1,...,8,10,3,10,8,10,5,2,5,12
4,3,1,2,2,1,2,1,2,1,2,...,8,4,2,7,8,3,4,2,4,22
5,8,2,11,1,1,11,2,1,1,1,...,10,8,13,11,10,21,3,1,8,20
6,10,1,3,3,1,3,1,5,1,2,...,21,10,7,3,21,11,4,5,4,17
7,5,3,3,3,3,9,3,7,4,3,...,17,3,3,9,17,10,5,3,5,24
8,2,2,2,1,1,2,1,2,5,2,...,3,2,7,2,3,2,1,2,1,17
9,5,1,1,1,1,1,1,1,2,1,...,19,17,1,8,19,5,3,1,3,11


We finally have our signature matrix. Now, we need to work on grouping similar users, using this matrix.

3. **Buckets**: once we have our signature matrix we need to put users with similar interests in the same bucket. To do that we can divide the signature matrix into bands, each consisting of a specified number of rows. Each band undergoes separate hashing. In this instance, we have chosen a band size (b) of 4, indicating that users sharing the same first 3 rows will be regarded as similar. As we increase the value of b, the likelihood diminishes that another paper will exhibit an identical permutation. What we are really looking for when we select the band size is for our tolerance for false positives (no similar users ending up in the same bucket) and false negatives (similar users not ending up in the same bucket).

Now let's write a function to split users into buckets.

In [15]:
def bucket(signature_matrix, num_band):
    # Create a dictionary to represent buckets
    buckets = dict()

    # Transpose the signature matrix
    sign_matrix = signature_matrix.T

    # Iterate through each band
    for n in range(num_band):
        # Calculate the number of columns in each band
        num_col = (sign_matrix.shape[1] // num_band)

        # Extract the part of the signature matrix corresponding to the current band
        part = sign_matrix.iloc[:, (num_col * n):num_col * (n + 1)]

        # Initialize a counter for row indexing
        a = 0

        # Iterate through each row in the current band
        for row_index in range(part.shape[0]):
            # Extract the signature values for the current row
            signature_tuple = tuple(part.iloc[row_index])

            # Check if the signature tuple is already a key in the buckets dictionary
            if signature_tuple not in buckets.keys():
                # If not, create a new entry with a list containing the current row index
                buckets[signature_tuple] = list()
                buckets[signature_tuple].append(part.index[a])
            else:
                # If the key already exists, append the current row index to the existing list
                buckets[signature_tuple].append(part.index[a])

            a += 1

    return buckets


In [16]:
buckets = bucket(signature_matrix, num_band = 4)
len(buckets)

1248

This is the number of different buckets. Each user is in a bucket with users with similar interests. This is our first filter to identify similarities.


### 1.3 Locality-Sensitive Hashing (LSH)

Now that your buckets are ready, it's time to ask a few queries. We will provide you with some user_ids and ask you to recommend at __most five movies__ to the user to watch based on the movies clicked by similar users.

To recommend at most five movies given a user_id, use the following procedure:

1. Identify the <ins>two most similar</ins> users to this user.
2. If these two users have any movies __in common__, recommend those movies based on the total number of clicks by these users.
3. If there are __no more common__ movies, try to propose the most clicked movies by the __most similar user first__, followed by the other user.

__Note:__ At the end of the process, we expect to see at most five movies recommended to the user.

__Example:__ assume you've identified user __A__ and __B__ as the most similar users to a single user, and we have the following records on these users:

- User A with 80% similarity
- User B with 50% similarity
  
|user|movie title|#clicks|
|---|---|---|
|A|Wild Child|20|
|A|Innocence|10|
|A|Coin Heist|2|
|B|Innocence|30|
|B|Coin Heist|15|
|B|Before I Fall|30|
|B|Beyond Skyline|8|
|B|The Amazing Spider-Man|5|

- __Recommended movies__ in order:
    - Innocence
    - Coin Heist
    - Wild Child
    - Before I Fall
    - Beyond Skyline

---

Once we have obtained the buckets with users organized based on common interests, the task requires taking a specific user_id as input and returning a maximum of 5 movies to recommend, leveraging other users with similar interests.

To address this task, we propose selecting all users from the bucket(s) where the input user_id is present and calculating the Jaccard similarity between this user and others in the same bucket(s). This process results in a matrix consisting of one row and as many columns as users compared to the input user. Subsequently, the two users with the highest similarity are selected, as requested, and a decision tree is implemented to recommend movies.

In summary, the approach involves identifying users with similar interests, measuring their similarity, and utilizing the two most similar users to guide the movie recommendations through a decision tree.

In [17]:
# user_id = str(input("Insert user id:"))
user_id = 'b15926c011'

Let's define some useful functions to find users in the same input user bucket and compute the jaccard similarity with them.

In [18]:
# Function to find keys in a dictionary associated with a specific value
def find_keys(dizionario, obj):
    keys = []
    for key, val in dizionario.items():
        if obj in val:
            for value in val:
                if value != obj:
                    keys.append(value)
    return set(keys)

# Function to calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

# Function to create a Jaccard similarity matrix for a given user ID
def jaccard_matrix(user_id, buckets, user_genres):
    chiavi_associate = find_keys(buckets, user_id)
    jm = pd.DataFrame(index = [user_id], columns = list(chiavi_associate))
    
    # Calculate Jaccard similarity for each associated user
    for i in jm.columns:
        
        # Populate the DataFrame with Jaccard similarity values
        jm.loc[user_id, i] = jaccard_similarity(user_genres[user_id], user_genres[i])
    return jm

jm = jaccard_matrix(user_id = "b15926c011", buckets=buckets, user_genres = user_genres)
jm

,310a683456,a92c94f8a6,d84ea4be48,d01654940f,f68264266b,b84046f8d0,84377d709d,852049ca55,262247f204,4c8e496219,...,36e6d036e0,2187327cdb,bd83c291ae,62cb7555f3,e03bbdd786,e8682710b1,e730939d8a,4466ebee7a,c25470e0a7,3d43158635
b15926c011,0.0,0.235294,0.0,0.368421,0.526316,0.35,0.4,0.473684,0.352941,0.666667,...,0.647059,0.764706,0.176471,0.588235,0.526316,0.444444,0.176471,0.0,0.411765,0.631579


Let's extract the 2 most similar users to the input one.

In [19]:
jm = jm.astype(float)
# select the 2 biggest values
most_similar_users, values_sim = list(jm.squeeze().nlargest(2).index), list(jm.squeeze().nlargest(2))
dic = dict()

# select the 2 most similar users
for user, similarity in zip(most_similar_users, values_sim):
    dic[user] = similarity
dic

{'779343a3ea': 1.0, '1daab4d5ce': 0.9444444444444444}

Now let's join all together in a single function, which recommends films for a given user id.

In [25]:
# Function to recommend films for a given user ID

def recommended_films(jm, user_id, buckets, user_genres):
    
    # Calculate Jaccard similarity matrix for the given user ID
    jm = jaccard_matrix(user_id = user_id, buckets=buckets, user_genres = user_genres)

    jm = jm.astype(float)
    
    # Find the most similar users based on Jaccard similarity
    most_similar_users, values_sim = list(jm.squeeze().nlargest(2).index), list(jm.squeeze().nlargest(2)) # select the 2 biggest values
    dic = dict()
    
    # select the 2 most similar users
    for user, similarity in zip(most_similar_users, values_sim):
        dic[user] = similarity

    # Group and sort movies based on user clicks
    movie_counts = df[df['user_id'].isin(most_similar_users)].groupby(['user_id', 'movie_id']).size().reset_index(name='clicks')
    df_sorted = movie_counts.sort_values(by=['user_id', 'clicks'], ascending=[True, False])
    top_10_movies_per_user = df_sorted.groupby('user_id').head(10000)

    # Initialize a list to store recommended films
    recommended_films = []
    
    # Find duplicated movies in the top 10 list
    dupl = top_10_movies_per_user[top_10_movies_per_user.duplicated('movie_id', keep=False)].groupby(['movie_id']).size().reset_index(name='clicks').head(5)
    
    # Sort duplicated movies based on clicks and add to recommended_films list
    app = dupl.sort_values(by = "clicks").movie_id
    recommended_films.extend(app.tolist())
    
    # Iterate through most similar users
    for i in most_similar_users:
        subset = top_10_movies_per_user[top_10_movies_per_user['user_id'] == i].head(5)
        for index, row in subset.iterrows():
            if len(recommended_films) < 5:
                recommended_films.append(row.movie_id)
            else:
                return recommended_films
    return recommended_films

# Function to retrieve movie details for recommended films
def lsh(jm, user_id, buckets, user_genres, df):
    # Get the list of recommended films for the given user ID
    idx = recommended_films(jm, user_id = user_id, buckets = buckets, user_genres = user_genres)
    res = []
    
    # Iterate through recommended movie indices and fetch details
    for i in idx:
        row = df[df['movie_id'] == i][['movie_id', 'title']].iloc[0]
        res.append(row)

    out = pd.DataFrame(res)

    return out

In [26]:
idx = recommended_films(jm, user_id = user_id, buckets = buckets, user_genres = user_genres)
# print(idx)
films = lsh(jm, user_id = user_id, buckets = buckets, user_genres = user_genres, df = df)
films

,movie_id,title
1281,e52ac4e2fa,Audrie & Daisy
56607,e63017b477,"Love, Rosie"
2437,57e2731b38,Coin Heist
371322,a215824200,The Edge of Seventeen
15408,428552c66d,Minor Details


## 5. Algoritmic Question (AQ)
Federico studies in a demanding university where he has to take a certain number N of exams to graduate, but he is free to choose in which order he will take these exams. Federico is panicking since this university is not only one of the toughest in the world but also one of the weirdest. His final grade won't depend at all on the mark he gets in these courses: there's a precise evaluation system.

He was given an initial personal score of S when he enrolled, which changes every time he takes an exam: now comes the crazy part. He soon discovered that every of the N exams he has to take is assigned a mark p. Once he has chosen an exam, his score becomes equal to the mark p, and at the same time, the scoring system changes:

*   If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by the quantity S-p
*   If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by the quantity p-S

So, for example, consider S=8 as the initial personal score. Federico must decide which exam he wants to take, being [5,7,1] the marks list. If he takes the first one, being 5<8 and 8-5=3, the remaining list now becomes [10, 4], and his score is updated as S=5.

In this chaotic university where the only real exam seems to be choosing the best way to take exams, you are the poor student advisor who is facing a long queue of confused people who need some help. Federico is next in line, and he comes up in turn with an inescapable question: he wants to know which is the highest score possible he could get.

In [22]:
#Take the inputs.
initial_score = int(input())
marks = list(map(int, input().split()))

ValueError: invalid literal for int() with base 10: 'st'

a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on because he wants to do paid counseling for other desperate students: in a recursive fashion, the helped helps the helpable.

In [ ]:
# In order to find the maximum score possible, we try every alternative order we can take the exams.
# To do this, we use the permutation function from itertools library and save the results to a list. Then, we consider every
# possible order one by one and see what is the biggest score the student can get.

all_permutations = permutations(marks)

# To be able to save the score we find when we consider a spesific solution, we create an empty list called possible_scores.
possible_scores = list()

# Iterate over all possible ways we can take the exams.
for perm in all_permutations:
  # Since the remaining marks change after each selection, we keep our current marks in a list we update after each selection.
  temp_marks = perm
  # Take the initial score as the current_score.
  current_score = initial_score
  # Iterate all the elements in the temp_marks list.
  for i in range(len(temp_marks)-1):
    # Take the first exam.
    selected_mark = temp_marks[0]
    # Calculate the difference between the exam's score and the current score of Federico.
    a = abs(selected_mark-current_score)
    # Delete the exam we have selected from the list so that we don't take the same exam again.
    temp_marks = temp_marks[1:]
    # Calculate the new exam scores for the remaining marks.
    if (selected_mark<current_score):
      temp_marks = [x + a for x in temp_marks]
    else:
      temp_marks = [x - a for x in temp_marks]
    # Update current_score as the mark of the selected exam.
    current_score = selected_mark
  # After going through all the exams in one of the permutations, save the score.
  # Since all other items in the temp_marks is deleted after being chosen, the last remaining mark in the list will be the final score for that permutation element.
  last_score = temp_marks[0]
  # Save the score into the possible_scores list.
  possible_scores.append(last_score)
# The maximum score Federico can get is the maximum in the possible scores list.
print (max(possible_scores))

11


b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university!

In [ ]:
all_permutations = permutations(marks)
possible_scores = list()

for perm in all_permutations:
  temp_marks = perm
  current_score = initial_score
  for i in range(len(temp_marks)-1):
    selected_mark = temp_marks[0]
    a = abs(selected_mark-current_score)
    temp_marks = temp_marks[1:]
    if (selected_mark<current_score):
      temp_marks = [x + a for x in temp_marks]
    else:
      temp_marks = [x - a for x in temp_marks]
    current_score = selected_mark
  last_score = temp_marks[0]
  possible_scores.append(last_score)

print ("Time complexity is dominated by the permutation operation and the nested for loops. So, the time complexity is O(N!*N). Federico is right, the code is slow especially if there are long lists of exams Federico can take.")

Time complexity is dominated by the permutation operation and the nested for loops. So, the time complexity is O(N!*N). Federico is right, the code is slow especially if there are long lists of exams Federico can take.


c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

In [ ]:
# In the following method in order to get rid off the permutations, which increase the time complexity the most, a recursive function is used.
def max_score_calculator(initial_score, marks):
    # To be able to save the score we find when we consider a spesific solution, we create an empty list called possible_scores.
    possible_scores = list()
    # Iterate all the elements in the marks list.
    for i in range(len(marks)):
        # Select the marks one by one in each iteration.
        selected_mark = marks[i]
        # Calculate the difference between the exam's score and the current score of Federico.
        a = abs(selected_mark - initial_score)
        # Delete the mark of the selected exams from the marks list.
        remaining_marks = marks[:i] + marks[i+1:]
        # Update current_score as the mark of the selected exam.
        if selected_mark < initial_score:
            updated_marks = [x + a for x in remaining_marks]
        else:
            updated_marks = [x - a for x in remaining_marks]
        # Append the result to the list of possible_scores while recursively calling the function with the selected_mark and updated_marks.
        possible_scores.append(max_score_calculator(selected_mark, updated_marks))

    # If the possible_scores list is empty, return the initial_score. If not, return the maximum element in the possible_scores list.
    if not possible_scores:
      return initial_score
    else:
      return max(possible_scores)

print (max_score_calculator(initial_score, marks))
print ("Time complexity is dominated by the recursive call inside the for loop. The recursive call will be exacuted N-1 times in the worst case scenario. For loop will be executed N times. \nThus, the time complexity is O(N^2). Although this depends on the length of the marks list, it has improved.")

11
Time complexity is dominated by the recursive call inside the for loop. The recursive call will be exacuted N-1 times in the worst case scenario. For loop will be executed N times. 
Thus, the time complexity is O(N^2). Although this depends on the length of the marks list, it has improved.


d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!

In [ ]:
# I provided ChatGPT 3.5 the code I wrote in the c) part and told it to "write an improved version of the following code, especially focusing on improving the time complexity".
# After it provided the code, I asked it to comment on every line. Here is the resulting code:

# Define a function 'max_score_calculator' that takes initial_score and marks as parameters
def max_score_calculator(initial_score, marks):
    # Define an inner function 'calculate_max_score' that takes current_score and remaining_marks as parameters
    def calculate_max_score(current_score, remaining_marks):
        # If there are no remaining marks, return the current_score
        if not remaining_marks:
            return current_score

        # Initialize a variable to keep track of the maximum possible score
        max_possible_score = float('-inf')

        # Iterate through each mark in the remaining_marks list
        for i in range(len(remaining_marks)):
            # Get the mark at the current index
            selected_mark = remaining_marks[i]
            # Calculate the absolute difference between the selected_mark and the current_score
            a = abs(selected_mark - current_score)
            # Create a new list excluding the current mark from the remaining_marks
            updated_marks = remaining_marks[:i] + remaining_marks[i+1:]

            # Update the updated_marks list based on the comparison between selected_mark and current_score
            if selected_mark < current_score:
                # Adjust each mark in updated_marks by adding the absolute difference
                updated_marks = [x + a for x in updated_marks]
                # Recursively call the function with the updated score and marks
                score = calculate_max_score(selected_mark, updated_marks)
            else:
                # Adjust each mark in updated_marks by subtracting the absolute difference
                updated_marks = [x - a for x in updated_marks]
                # Recursively call the function with the updated score and marks
                score = calculate_max_score(selected_mark, updated_marks)

            # Update the maximum possible score considering the current calculated score
            max_possible_score = max(max_possible_score, score)

        # Return the maximum possible score after iterating through all marks
        return max_possible_score

    # Start the calculation by calling the inner function with initial_score and marks
    return calculate_max_score(initial_score, marks)

# Call the max_score_calculator function with initial_score and marks, then print the result
print(max_score_calculator(initial_score, marks))
print ("The time complexity of this code is again, O(N^2) to the recursion inside the loop and due to the worst-case scenario approach. \n However, this method uses memoization to store the results of intermediate computations. This allows the function to avoid recalculating the same scores multiple times. Thus, this version is more efficient.")

11
The time complexity of this code is again, O(N^2) to the recursion inside the loop and due to the worst-case scenario approach. 
 However, this method uses memoization to store the results of intermediate computations. This allows the function to avoid recalculating the same scores multiple times. Thus, this version is more efficient.
